IMDb (Internet Movie Database) est une base de données en ligne populaire contenant des informations sur les films, les séries télévisées et les jeux vidéo. Les ensembles de données IMDb font souvent référence à des ensembles de données dérivés d'IMDb, qui peuvent inclure des informations sur les films, telles que les titres, les genres, les dates de sortie et les évaluations des utilisateurs.

La mise au point d'un modèle BERT (Bidirectional Encoder Representations from Transformers) implique l'entraînement d'un modèle BERT pré-entraîné sur une tâche ou un ensemble de données spécifique afin de l'adapter aux exigences particulières de cette tâche. BERT est un puissant modèle de représentation du langage pré-entraîné qui peut être affiné pour diverses tâches de traitement du langage naturel (NLP), notamment l'analyse des sentiments, la réponse aux questions et la classification des textes.

Voici un aperçu général du processus d'ajustement d'un modèle BERT sur les ensembles de données IMDb :

1. **Préparation des données :**
   - Obtenez le jeu de données IMDb, qui peut inclure des critiques de films et des étiquettes de sentiment associées (positif ou négatif).
   - Prétraitez les données textuelles, y compris la tokenisation, le padding et la conversion du texte en représentations numériques.

   Exemple de code :

   ```python
   # Code pour charger et prétraiter le jeu de données IMDb
   import pandas as pd
   from sklearn.model_selection import train_test_split

   # Charger le jeu de données IMDb
   imdb_data = pd.read_csv('chemin/vers/imdb_data.csv')

   # Diviser les données en ensembles d'entraînement et de validation
   train_data, validation_data = train_test_split(imdb_data, test_size=0.2, random_state=42)

   # Autres étapes de prétraitement (tokenisation, padding, etc.)
   ```

2. **Modèle BERT et Tokenizer :**
   - Choisissez un modèle BERT pré-entraîné (tel que BERT-base ou BERT-large) et son tokenizer correspondant. Utilisez des bibliothèques populaires comme Hugging Face's Transformers en Python pour accéder aux modèles BERT pré-entraînés.

   Exemple de code :

   ```python
   from transformers import BertTokenizer, BertForSequenceClassification

   # Charger le tokenizer et le modèle BERT
   tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
   model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

   # Modifier ou ajouter une couche de classification pour la tâche spécifique
   ```

3. **Affinage :**
   - Chargez le modèle BERT pré-entraîné et remplacez ou ajoutez une couche de classification pour la tâche spécifique (par exemple, l'analyse de sentiment).
   - Définissez une boucle d'entraînement pour affiner le modèle sur le jeu de données IMDb. Cela implique d'ajuster les poids du modèle en fonction du jeu de données pour améliorer ses performances sur la tâche spécifique.

   Exemple de code :

   ```python
   from transformers import AdamW
   from torch.utils.data import DataLoader

   # Définir l'optimiseur et la fonction de perte
   optimizer = AdamW(model.parameters(), lr=1e-5)
   loss_function = ...

   # Définir la boucle d'entraînement
   for epoch in range(num_epochs):
       for batch in DataLoader(train_data, batch_size=batch_size, shuffle=True):
           # Autres étapes d'entraînement
   ```

4. **Optimisation des hyperparamètres avec Optuna :**
   - Optuna est un framework d'optimisation des hyperparamètres qui peut être utilisé pour trouver les hyperparamètres optimaux pour votre modèle.
   - Définissez les hyperparamètres à optimiser, tels que le taux d'apprentissage, la taille du lot (batch size) et le taux de dropout.
   - Utilisez Optuna pour exécuter plusieurs essais avec différentes configurations d'hyperparamètres, évaluant les performances du modèle sur un ensemble de validation pour chaque essai.
   - Optuna recherchera automatiquement l'espace des hyperparamètres pour trouver la configuration qui conduit aux meilleures performances du modèle.

   Exemple de code :

   ```python
   import optuna

   def objectif(trial):
       learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2)
       batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
       dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)

       # Définir et entraîner votre modèle BERT avec les hyperparamètres spécifiés

       # Évaluer le modèle sur un ensemble de validation et retourner la métrique de performance
       accuracy = evaluate_model(model, validation_data)

       return accuracy

   # Créer un objet d'étude et optimiser la fonction objectif
   study = optuna.create_study(direction='maximize')
   study.optimize(objectif, n_trials=10)

   # Obtenir les meilleurs hyperparamètres
   best_params = study.best_params
   ```

Ces exemples vous donneront une base pour commencer à affiner un modèle BERT sur le jeu de données IMDb en utilisant Optuna pour l'optimisation des hyperparamètres.

### 1. Importation d'un ensemble de données

In [ ]:
!gdown --id 1fkW51Bd1WQy52U9Yg9dn2Gk67V4_n5GA

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1fkW51Bd1WQy52U9Yg9dn2Gk67V4_n5GA
To: /content/IMDB-Dataset.csv
100% 66.2M/66.2M [00:01<00:00, 53.9MB/s]


In [ ]:
import numpy as np
import pandas as pd
import re
import string
import torch
from string import digits


In [ ]:
df = pd.read_csv('IMDB-Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


### 2. Diviser les données en un ensemble de formation, un ensemble de test et un ensemble d'essai.

In [ ]:
# Diviser l'ensemble de données en train, test et dev
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

### 3. Prétraitement des données

In [ ]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 5.8 MB/s eta 0:00:00


In [ ]:

import contractions
from bs4 import BeautifulSoup

def preprocess_text(text):
    text = BeautifulSoup(text, "html.parser").get_text()
    text = text.lower()
    text = contractions.fix(text)
    text = re.sub(r'[^a-zA-Z.!?]+', ' ', text)
    return text

train_df['review'] = train_df['review'].apply(preprocess_text)
val_df['review'] = val_df['review'].apply(preprocess_text)
test_df['review'] = test_df['review'].apply(preprocess_text)

<ipython-input-6-8e7a9b096c21>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.3 MB/s eta 0:00:00


# 4. Importation du tokenizer BERT

In [ ]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
initial_state_dict = model.state_dict()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-mini and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def encode_data(tokenizer, text, max_length):
    tokens = tokenizer.batch_encode_plus(
        text,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    return tokens['input_ids'], tokens['attention_mask']
max_length = 256

train_ids, train_masks = encode_data(tokenizer, train_df['review'].tolist(), max_length)
val_ids, val_masks = encode_data(tokenizer, val_df['review'].tolist(), max_length)
test_ids, test_masks = encode_data(tokenizer, test_df['review'].tolist(), max_length)

train_labels = torch.tensor(train_df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0).values)
val_labels = torch.tensor(val_df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0).values)
test_labels = torch.tensor(test_df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0).values)

### 5. Implémentation des chargeurs de données pour les ensembles de formation, de test et de développement

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 256

train_data = TensorDataset(train_ids, train_masks, train_labels)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

val_data = TensorDataset(val_ids, val_masks, val_labels)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

test_data = TensorDataset(test_ids, test_masks, test_labels)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

### 7. Définir la fonction objectif

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
num_epochs=16
model.load_state_dict(initial_state_dict)

optimizer = AdamW(model.parameters())
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(0.1 * total_steps), num_training_steps=total_steps)

best_val_loss = float('inf')
model.to(device)

for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1} (Training)", ncols=100):
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs[0]
            loss.backward()

            optimizer.step()
            scheduler.step()

            train_loss += loss.item()

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Epoch {epoch+1} (Validation)", ncols=100):
                input_ids, attention_mask, labels = batch
                input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs[0]

                val_loss += loss.item()

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state_dict=model.state_dict()



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 9 (Training):   8%|███▎                                      | 10/125 [00:06<01:10,  1.62it/s]


KeyboardInterrupt: ignored

### 10. Évaluer le modèle sur l'ensemble de test

In [ ]:
from sklearn.metrics import accuracy_score, classification_report



model.load_state_dict(best_model_state_dict)
model.eval()

predictions, true_labels = [], []

with torch.no_grad():
        for batch in test_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs[0]

            _, predicted_labels = torch.max(logits, 1)

            predictions.extend(predicted_labels.cpu().numpy().tolist())
            true_labels.extend(labels.cpu().numpy().tolist())

accuracy = accuracy_score(true_labels, predictions)
classification_report = classification_report(true_labels, predictions, target_names=['negative', 'positive'])


print(f'Précision: {accuracy:.4f}')
print('Rapport de classification :')
print(classification_report)

Précision: 0.8695
Rapport de classification :
              precision    recall  f1-score   support

    negative       0.86      0.88      0.87      4961
    positive       0.88      0.86      0.87      5039

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



# Testez sur votre propre texte

In [ ]:
# To calculate the loss, we need to pass in a label:
input_str='holy Sh*t this was god awful. i sat in the theater for for an hour and ten minutes and i thought i was going to gouge out my eyes much in the manor Oedipus Rex. dear god. this movie deserves no more credit than anything done by a middle school'
model_inputs = tokenizer(input_str, return_tensors="pt")

labels = ['NEGATIVE', 'POSITIVE']
model_inputs['labels'] = torch.tensor([0])

for key in model_inputs.keys():
  model_inputs[key]=model_inputs[key].cuda()
with torch.no_grad():
  model_outputs = model(**model_inputs)


print(model_outputs)
print()
print(f"Model predictions: {labels[model_outputs.logits.argmax()]}")

SequenceClassifierOutput(loss=tensor(7.5834, device='cuda:0'), logits=tensor([[ 3.3463, -4.2366]], device='cuda:0'), hidden_states=None, attentions=None)

Model predictions: NEGATIVE
